In [1]:
import numpy as np                       
import xarray as xr                      
import pandas as pd     
import cftime                  
from datetime import datetime, timedelta
import os

## Prepare the training data

In [2]:
reanalysis_nc = xr.open_dataset('../Archivos_reanalisisERA5_realesINUMET/Reanalisis/Datos_t2m_horario_2000a2010_uy.nc')
reanalysis_nc_2 = xr.open_dataset('../Archivos_reanalisisERA5_realesINUMET/Reanalisis/Datos_t2m_horario_2011a2021_uy.nc')

def transform_reanalysis(data_nc):
    reanalysis = pd.DataFrame()
    reanalysis['time'] = pd.to_datetime(data_nc.time)
    reanalysis['temp'] = data_nc.t2m.mean(dim=["latitude","longitude"]) 
    reanalysis = reanalysis.loc[reanalysis["time"] < datetime.fromisoformat('2015-01-01T00:00:00')]
    reanalysis['date'] = reanalysis['time'].dt.date
    reanalysis['hour'] = reanalysis['time'].dt.hour
    reanalysis['day'] = reanalysis['time'].dt.day
    reanalysis['month'] = reanalysis['time'].dt.month
    reanalysis_by_day = reanalysis[["date", "temp"]].groupby('date') \
                                                   .mean() \
                                                   .rename(columns={"temp":"avg_temp"})
    reanalysis = reanalysis.join(reanalysis_by_day, on="date") \
                            .drop('date', axis=1)
    reanalysis = reanalysis[['time', 'hour', 'day', 'month', 'avg_temp', 'temp']] #Reorder the columns
    return reanalysis

reanalysis = pd.concat([transform_reanalysis(reanalysis_nc), transform_reanalysis(reanalysis_nc_2)])
reanalysis = reanalysis[((reanalysis['day'] != 29) | (reanalysis['month'] != 2))] #Delete all Feb 29th 
reanalysis.to_csv('reanalysis_training_data.csv',  index=False)

## Prepare the data to be predicted

In [3]:
CESM2_WACCM_nc = xr.open_dataset('../data/tas_day_CESM2-WACCM_historical_r1i1p1f1_gn_20000101-20141231_v20190227.nc')
CESM2_WACCM = pd.DataFrame()
CESM2_WACCM['time'] = CESM2_WACCM_nc.time
CESM2_WACCM['CESM2_WACCM'] = CESM2_WACCM_nc.tas.mean(dim=["lat","lon"])
CESM2_WACCM['time'] = CESM2_WACCM['time'].apply(lambda x: datetime.fromisoformat(x.isoformat()))

CESM2_WACCM_by_hour = []

for row in CESM2_WACCM.values:
    for h in range(0,24):
        new_row = pd.Series({
            'time': row[0] + timedelta(hours=h),
            'hour': h,
            'day': row[0].day,
            'month': row[0].month,
            'avg_temp': row[1]
        })
        CESM2_WACCM_by_hour.append(new_row)

CESM2_WACCM = pd.DataFrame(CESM2_WACCM_by_hour)
CESM2_WACCM.to_csv('cmip6_to_predict_data.csv', index=False)

In [4]:
reanalysis.to_csv('reanalysis_training_data.csv',  index=False)